## Note:
this is a workbook notebook for testing the cnn model... the final notebook will have much more examples and will have visualization on how the data looks

In [28]:
# General imports
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
import random


#keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate

# Custom functions
%load_ext autoreload
%autoreload 2
import database_selection
import vectorization
import helpers
import icd9_cnn_model


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
#reading file
full_df = pd.read_csv('../data/disch_notes_all_icd9.csv',
                 names = ['HADM_ID', 'SUBJECT_ID', 'DATE', 'ICD9','TEXT'])


In [32]:
full_df.shape

(52696, 5)

In [41]:
df = full_df.sample(frac=0.1).reset_index(drop=True)
print df.shape
df.head(10)

(5270, 5)


,HADM_ID,SUBJECT_ID,DATE,ICD9,TEXT
0,181955,41710,2133-11-01 00:00:00,99604 2763 42822 99811 2851 4254 2875 4280 414...,Admission Date: [**2133-10-29**] ...
1,178527,66643,2190-11-21 00:00:00,99831 5570 0389 99592 78552 9982 5119 78959 27...,Admission Date: [**2190-11-2**] ...
2,182178,85085,2171-07-31 00:00:00,51181 4829 4280 51881 1961 28982 4928 4589 162...,Admission Date: [**2171-7-23**] ...
3,191112,17789,2123-03-19 00:00:00,41401 4280 4400 25000 60001 4019 2720,"Name: [**Known lastname 11249**],[**Known fir..."
4,174261,5690,2136-05-15 00:00:00,44102 5859 4400 5738 4589 04183 40390,Admission Date: [**2136-5-11**] ...
5,101251,32583,2170-03-03 00:00:00,48241 5849 1628 79902 496 4019 2724 7242,Admission Date: [**2170-2-25**] ...
6,139743,48705,2146-08-04 00:00:00,80708 8600 86120 8054 78009 4589 29680,Admission Date: [**2146-7-31**] ...
7,186478,28316,2156-08-25 00:00:00,7742 76516 76525 77081 76070,Admission Date: [**2156-8-16**] Dischar...
8,168833,46039,2136-05-16 00:00:00,3962 3970 2875 42731 496 4019 60000 6929,Admission Date: [**2136-5-9**] D...
9,187383,27064,2181-12-29 00:00:00,7661 7706 7746,Admission Date: [**2181-12-25**] Discha...


In [3]:
# instead of finding out the top 20 leave icd-9 codes and filter records based on that
# we will use all records and replace the leave icd-9 codes with its grandparents code in the first level of the hierarchy
#N_TOP = 20 
#full_df, top_codes = database_selection.filter_top_codes(df, 'ICD9', N_TOP, filter_empty = True)
#df = full_df.head(1000)

### Replacing leave icd-9 codes with their grandparent icd-9 code in the first level of the hierarchy

Source: https://github.com/sirrice/icd9   
The code above let's you see the ICD-9 hierarchy and traverse it, getting the parents (path) of a node, the children of a node, siblings, etc. (well documented in its README file).  

From looking at the top of the hierarchy, these are the ICD9-codes that are in the first level of the hierarchy.
```
001-139 INFECTIOUS AND PARASITIC DISEASES 
140-239 NEOPLASMS 
240-279 ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS 
290-319 MENTAL DISORDERS 
320-389 DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS 
390-459 DISEASES OF THE CIRCULATORY SYSTEM 
460-519 DISEASES OF THE RESPIRATORY SYSTEM 
520-579 DISEASES OF THE DIGESTIVE SYSTEM 
580-629 DISEASES OF THE GENITOURINARY SYSTEM 
630-679 COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM 
680-709 DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE 
710-739 DISEASES OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE 
760-779 CERTAIN CONDITIONS ORIGINATING IN THE PERINATAL PERIOD 
780-789 SYMPTOMS 
790-796 NONSPECIFIC ABNORMAL FINDINGS 
797 Senility without mention of psychosis
798 Sudden death, cause unknown
799 Other ill-defined and unknown causes of morbidity and mortality
800-999 INJURY AND POISONING 
```

The way that ICD9-codes are coded makes easy to find out which icd9-code code is the granparent in the first level,
for example:
```
leave-code  code-at-first-level
64833    -> 630-679
4019     -> 390-459
```

The first three charachters of the leave icd9-code can be used to find out which is the grandparent icd-code in the first level

In [52]:
# this function will go to database_selection.py 
ICD9_FIRST_LEVEL = [
    '001-139','140-239','240-279','290-319', '320-389', '390-459','460-519', '520-579', '580-629', 
    '630-679', '680-709','710-739', '760-779', '780-789', '790-796', '797', '798', '799', '800-999' ]
ICD9_RANGES = [x.split('-') for x in ICD9_FIRST_LEVEL]
N_TOP = len(ICD9_FIRST_LEVEL)
def replace_with_grandparent_codes(string_codes):
    """ Creates a sring of the codes which are both in the original string
        and in the top codes list """
    resulting_codes = []
    for code in string_codes.split(' '):
        for i,gparent_range in enumerate(ICD9_RANGES):
            range = gparent_range[1] if len(gparent_range) == 2 else gparent_range[0]
            if code[0:3] <= range:
                resulting_codes.append(ICD9_FIRST_LEVEL[i])
                break 
    return ' '.join (set(resulting_codes))

In [54]:
df['ICD9'] = df['ICD9'].apply(lambda x: replace_with_grandparent_codes(x))
df.head(10)

,HADM_ID,SUBJECT_ID,DATE,ICD9,TEXT
0,181955,41710,2133-11-01 00:00:00,240-279 390-459 290-319 460-519 800-999 580-629,Admission Date: [**2133-10-29**] ...
1,178527,66643,2190-11-21 00:00:00,240-279 001-139 390-459 290-319 460-519 520-57...,Admission Date: [**2190-11-2**] ...
2,182178,85085,2171-07-31 00:00:00,240-279 760-779 390-459 290-319 460-519 140-23...,Admission Date: [**2171-7-23**] ...
3,191112,17789,2123-03-19 00:00:00,240-279 580-629 390-459,"Name: [**Known lastname 11249**],[**Known fir..."
4,174261,5690,2136-05-15 00:00:00,580-629 001-139 390-459 520-579,Admission Date: [**2136-5-11**] ...
5,101251,32583,2170-03-03 00:00:00,240-279 390-459 460-519 800-999 140-239 580-62...,Admission Date: [**2170-2-25**] ...
6,139743,48705,2146-08-04 00:00:00,780-789 390-459 290-319 800-999,Admission Date: [**2146-7-31**] ...
7,186478,28316,2156-08-25 00:00:00,760-779,Admission Date: [**2156-8-16**] Dischar...
8,168833,46039,2136-05-16 00:00:00,580-629 680-709 390-459 290-319 460-519,Admission Date: [**2136-5-9**] D...
9,187383,27064,2181-12-29 00:00:00,760-779,Admission Date: [**2181-12-25**] Discha...


## other pre-processing

In [56]:
#preprocess icd9 codes
top_codes = ICD9_FIRST_LEVEL
labels = vectorization.vectorize_icd_column(df, 'ICD9', top_codes)


In [58]:
labels[0:5]

array([[0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [57]:
#preprocess notes
MAX_VOCAB = None # to limit original number of words (None if no limit)
MAX_SEQ_LENGTH = 5000 # to limit length of word sequence (None if no limit)
df.TEXT = vectorization.clean_notes(df, 'TEXT')
data, dictionary, MAX_VOCAB = vectorization.vectorize_notes(df.TEXT, MAX_VOCAB, verbose = True)
data, MAX_SEQ_LENGTH = vectorization.pad_notes(data, MAX_SEQ_LENGTH)
print("Final Vocabulary: %s" % MAX_VOCAB)
print("Final Max Sequence Length: %s" % MAX_SEQ_LENGTH)

Vocabulary size: 44646
Average note length: 1621.08045541
Max note length: 8867
Final Vocabulary: 44646
Final Max Sequence Length: 5000


In [59]:
#split sets
X_train, X_val, X_test, y_train, y_val, y_test = helpers.train_val_test_split(
    data, labels, val_size=0.2, test_size=0.1, random_state=101)
print("Train: ", X_train.shape, y_train.shape)
print("Validation: ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

('Train: ', (3688, 5000), (3688, 19))
('Validation: ', (1054, 5000), (1054, 19))
('Test: ', (528, 5000), (528, 19))


In [60]:
# Delete temporary variables to free some memory
del df, data, labels

In [61]:
#creating embeddings
EMBEDDING_LOC = '../data/glove.6B.100d.txt' # location of embedding
EMBEDDING_DIM = 100 # given the glove that we chose
EMBEDDING_MATRIX, embedding_dict = vectorization.embedding_matrix(EMBEDDING_LOC,
                                                                  dictionary, EMBEDDING_DIM, verbose = True)


('Vocabulary in notes:', 44646)
('Vocabulary in original embedding:', 400000)
('Vocabulary intersection:', 21960)


## CNN for text classification

Based on the following papers and links:
* "Convolutional Neural Networks for Sentence Classification"   
* "A Sensitivity Analysis of (and Practitioners’ Guide to) Convolutional Neural Networks for Sentence Classification"
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://github.com/alexander-rakhlin/CNN-for-Sentence-Classification-in-Keras/blob/master/sentiment_cnn.py
* http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
* https://github.com/dennybritz/cnn-text-classification-tf/blob/master/text_cnn.py

In [62]:
reload(icd9_cnn_model)
#### build model
model = icd9_cnn_model.build_icd9_cnn_model (input_seq_length=MAX_SEQ_LENGTH, max_vocab = MAX_VOCAB,
                             external_embeddings = True, embedding_trainable =True,
                             embedding_dim=EMBEDDING_DIM,embedding_matrix=EMBEDDING_MATRIX,
                             num_filters = 100, filter_sizes=[2,3,4,5],
                             training_dropout_keep_prob=0.9,
                             num_classes=N_TOP )

In [63]:
# Train the model
model.fit(X_train, y_train, batch_size=50, epochs=5, validation_data=(X_val, y_val), verbose=2)

Train on 3688 samples, validate on 1054 samples
Epoch 1/5
138s - loss: 2.1384 - acc: 0.7246 - val_loss: 0.7414 - val_acc: 0.7317
Epoch 2/5
138s - loss: 1.1068 - acc: 0.7299 - val_loss: 0.7165 - val_acc: 0.7317
Epoch 3/5
130s - loss: 0.8495 - acc: 0.7292 - val_loss: 0.7138 - val_acc: 0.7317
Epoch 4/5
129s - loss: 0.7827 - acc: 0.7288 - val_loss: 0.7113 - val_acc: 0.7317
Epoch 5/5
129s - loss: 0.7580 - acc: 0.7292 - val_loss: 0.7079 - val_acc: 0.7319


In [64]:
pred_train = model.predict(X_train, batch_size=50)
pred_dev = model.predict(X_val, batch_size=50)

## Performance Evaluation

In [65]:
def get_f1_score(y_true,y_hat,threshold, average):
    hot_y = np.where(np.array(y_hat) > threshold, 1, 0)
    return f1_score(np.array(y_true), hot_y, average=average)

print 'F1 scores'
print 'threshold | training | dev  '
f1_score_average = 'micro'
for threshold in [ 0.02, 0.03,0.04,0.05,0.055,0.058,0.06, 0.08, 0.1,0.2,0.3, 0.5]:
    train_f1 = get_f1_score(y_train, pred_train,threshold,f1_score_average)
    dev_f1 = get_f1_score(y_val, pred_dev,threshold,f1_score_average)
    print '%1.3f:      %1.3f      %1.3f' % (threshold,train_f1, dev_f1)

F1 scores
threshold | training | dev  
0.020:      0.539      0.532
0.030:      0.574      0.563
0.040:      0.598      0.587
0.050:      0.620      0.601
0.055:      0.630      0.606
0.058:      0.634      0.611
0.060:      0.636      0.612
0.080:      0.615      0.584
0.100:      0.495      0.474
0.200:      0.035      0.032
0.300:      0.015      0.010
0.500:      0.001      0.001


### TO DO: model for Thresholding
Papers about Thresholding:    

* "Convolutional Neural Network using a Threshold Predictor for Multi-label Speech Act Classification"   
* "A Study on Threshold Selection for Multi-label Classification"   

These papers mention they use a model for thresholding, but it is not the main topic:   
* "A Review on Multi-Label Learning Algorithms"   
* "Large-scale Multi-label Text Classification—Revisiting Neural Networks"*   
* "A multi-label convolutional neural network for automatic image annotation"

### Results with external embeddings = True , no additional training,  top 20
```
F1 scores
threshold | training | dev  
0.020:      0.337      0.329
0.030:      0.360      0.353
0.040:      0.365      0.374
0.050:      0.372      0.375
0.055:      0.370      0.377
0.058:      0.369      0.375
0.060:      0.368      0.375
0.080:      0.348      0.361
0.100:      0.309      0.319
0.200:      0.198      0.208
0.300:      0.157      0.138
0.500:      0.000      0.000
```

### Results with external embeddings = False, top 20
```
F1 scores
threshold | training | dev  
0.020:      0.288      0.300
0.030:      0.327      0.322
0.040:      0.371      0.363
0.050:      0.380      0.391
0.055:      0.412      0.383
0.058:      0.403      0.394
0.060:      0.394      0.389
0.080:      0.385      0.390
0.100:      0.229      0.225
0.200:      0.000      0.000
0.300:      0.000      0.000
0.500:      0.000      0.000
```

### Results with external embedding and training them , top 20
```
F1 scores
threshold | training | dev  
0.020:      0.334      0.333
0.030:      0.362      0.360
0.040:      0.366      0.374
0.050:      0.373      0.380
0.055:      0.374      0.382
0.058:      0.376      0.376
0.060:      0.376      0.378
0.080:      0.387      0.371
0.100:      0.366      0.350
0.200:      0.179      0.171
0.300:      0.020      0.020
0.500:      0.000      0.000

```

### Results with external Embeddings = False, top 10, 
We can compare this setup with the LSTM published in the paper "Applying Deep Learning to ICD-9 Multi-label Classification from Medical Records", they got a F1-score of about 0.4168, we are getting 0.447

``` 
F1 scores
threshold | training | dev  
0.020:      0.399      0.407
0.030:      0.399      0.407
0.040:      0.399      0.407
0.050:      0.408      0.413
0.055:      0.433      0.420
0.058:      0.437      0.430
0.060:      0.432      0.427
0.080:      0.501      0.463
0.100:      0.446      0.463
0.200:      0.206      0.066
0.300:      0.000      0.000
0.500:      0.000      0.000
```



## Notes:


(1) There is a LSTM model by this paper: "Applying Deep Learning to ICD-9 Multi-label Classification from Medical Records" which did achieve a 42% F1-score. (https://cs224d.stanford.edu/reports/priyanka.pdf), but it only uses the top 10 icd9 codes. We are getting 46% (just running with 1000 notes so far)


(2) The "A Comparison of Rule-Based and Deep Learning Models for Patient Phenotyping"  study did get a 70% F1-score, but they don't use the icd9-labels but phenotypes labels they annotated themselved (via a group of medical professionals). (https://arxiv.org/abs/1703.08705). There were ONLY 10 phenotypes.

The discharge summaries are labeled with ICD9-codes that are leaves in the ICD9-hierarchy (which has hundreds of ICD9-codes), then maybe these leave nodes are too specific and difficult to predict, one experiment would be to replaced all the ICD9-codes with their parent in the second or third level in the hierarchy and see if predictions work better that way.   

(3) our baseline with top 20 codes had a f1-score of 35% (assigning top 4 icd9 codes to all notes, using a CNN with no external embeddings is getting about 40% f1-score.. a little better than the baseline

(4) Papers published and best practices report  that external embeddings improve considerable the model's performance.. maybe it is not the case here because of the medical terms..   

(5) Fixed Thresholding doesn't work well for multilabel classifications, we can implement a model to choose the appropriate threshold for each record (see notes above)